In [ ]:
from huggingface_hub import login
login(token="HF_TOKEN")

In [ ]:
!pip install diffusers -q

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("CharanSaiVaddi/negopt-sft-model")
model = AutoModelForSeq2SeqLM.from_pretrained("CharanSaiVaddi/negopt-sft-model")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from diffusers import StableDiffusionPipeline

# Helper to load an existing fine-tuned negative-prompt generator

def load_neg_generator(model, tokenizer):
    """
    Loads a tokenizer and seq2seq model from `model_path`.
    """
    tokenizer = tokenizer
    model = model
    return tokenizer, model

@torch.no_grad()
def generate_negative(
    prompt: str,
    tokenizer: AutoTokenizer,
    model: AutoModelForSeq2SeqLM,
    max_length: int = 64,
    num_beams: int = 4
) -> str:
    """
    Generates an optimized negative prompt for the given positive `prompt`.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True
    )
    neg_prompt = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return neg_prompt


# Function to produce images with and without negative prompts

def generate_images(
    pos_prompt: str,
    pipe: StableDiffusionPipeline,
    neg_generator: tuple,
    guidance_scale: float = 7.5,
    num_inference_steps: int = 50
):
    """
    Returns:
      - img_no_neg: image generated using only `pos_prompt`
      - img_with_neg: image generated using an optimized negative prompt
      - neg_prompt: the generated negative prompt string
    """
    tokenizer, model = neg_generator
    model.to(pipe.device)

    # Generate image without negative prompt
    img_no_neg = pipe(
        prompt=pos_prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps
    ).images[0]

    # Generate negative prompt and image
    neg_prompt = generate_negative(pos_prompt, tokenizer, model)
    img_with_neg = pipe(
        prompt=pos_prompt,
        negative_prompt=neg_prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps
    ).images[0]

    return img_no_neg, img_with_neg, neg_prompt


if __name__ == "__main__":
    # Determine device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load Stable Diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2"
    ).to(device)

    # Load the pre-fine-tuned neg-opt model
    neg_generator = load_neg_generator(model, tokenizer)

    # Example positive prompt
    pos_prompt = " Prompt: A Deer standing in the middle of a misty forest"

    # Generate images
    img_no_neg, img_with_neg, neg_prompt = generate_images(
        pos_prompt, pipe, neg_generator
    )

    # Save outputs
    img_no_neg.save("output_no_negative.png")
    img_with_neg.save("output_with_negative.png")
    print("Generated negative prompt:", neg_prompt)

In [ ]:
img_no_neg

In [ ]:
img_with_neg

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from typing import List, Tuple
import json

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms

#########################################
# Configuration Settings
#########################################
class Config:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_timesteps = 50  # Number of timesteps in reverse diffusion.
    # Quality threshold for the NIMA score (normalized to [0,1]).
    quality_threshold = 0.5
    output_dir = "collected_data_large"
    os.makedirs(output_dir, exist_ok=True)
    # Curated list of nature-themed prompts.
    prompts = [
        "a deer standing in the middle of a misty forest"
    ]
    # Path to NIMA checkpoint (if you have one)
    nima_checkpoint_path = "nima_checkpoint.pth"  # Update this if available

#########################################
# NIMA No-Reference Quality Assessment Model
#########################################
class NIMA(nn.Module):
    def __init__(self, base_model: str = 'vgg16', num_classes: int = 10):
        super(NIMA, self).__init__()
        self.num_classes = num_classes
        if base_model == 'vgg16':
            # Use VGG16 features. (You can experiment with other backbones.)
            vgg = models.vgg16(pretrained=True)
            self.features = vgg.features
            # Replace classifier with adaptive pooling and a fully-connected layer.
            self.pool = nn.AdaptiveAvgPool2d(1)
            self.dropout = nn.Dropout(0.75)
            self.fc = nn.Linear(512, num_classes)
        else:
            raise ValueError("Unsupported base model")

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)  # (B,512,1,1)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x  # Raw logits

    def predict_quality(self, x):
        """
        Returns the expected quality score for input x.
        The model outputs logits for scores 1..10 and we compute the expected value.
        """
        logits = self.forward(x)
        prob = F.softmax(logits, dim=1)
        # Scores 1 to 10.
        scores = torch.arange(1, self.num_classes + 1, device=x.device, dtype=torch.float)
        expected_quality = (prob * scores).sum(dim=1)
        return expected_quality

# Instantiate NIMA model and load checkpoint if available.
def load_nima_model(config: Config) -> NIMA:
    nima_model = NIMA(base_model='vgg16', num_classes=10).to(config.device)
    if os.path.exists(config.nima_checkpoint_path):
        checkpoint = torch.load(config.nima_checkpoint_path, map_location=config.device)
        nima_model.load_state_dict(checkpoint)
        print("Loaded NIMA checkpoint.")
    else:
        print("NIMA checkpoint not found; using base pretrained VGG16 features (not fine-tuned).")
    nima_model.eval()
    return nima_model

# Compute quality score using NIMA.
def compute_nima_quality(image: Image.Image, nima_model: NIMA, config: Config) -> float:
    # Define transforms to match what NIMA was trained on.
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    img_tensor = transform(image).unsqueeze(0).to(config.device)
    with torch.no_grad():
        quality = nima_model.predict_quality(img_tensor)
    # The expected quality score is between 1 and 10;
    # normalize it to a 0-1 scale by subtracting 1 and dividing by 9.
    quality_norm = (quality.item() - 1) / 9
    return quality_norm

#########################################
# Diffusion Simulator (RGB)
#########################################
class DiffusionSimulator:
    def __init__(self, config: Config):
        self.config = config

    def reverse_diffusion(self, latent: torch.Tensor, prompt: str = "") -> Tuple[Image.Image, List[torch.Tensor]]:
        """
        Simulate the reverse diffusion process (optionally conditioned on a prompt).
        Returns:
            generated_image: Final generated image (PIL RGB).
            noise_sequence: List of noise tensor predictions.
        """
        noise_sequence = []
        current_latent = latent.clone()
        for t in range(self.config.num_timesteps):
            # Simulated noise vector (random noise for demonstration).
            predicted_noise = torch.randn_like(current_latent)
            noise_sequence.append(predicted_noise.clone())
            # Simplified latent update.
            current_latent = current_latent - predicted_noise * 0.1
        generated_image = self.tensor_to_image(current_latent)
        return generated_image, noise_sequence

    @staticmethod
    def tensor_to_image(tensor: torch.Tensor) -> Image.Image:
        """
        Converts a tensor in the range [-1,1] to a PIL RGB image.
        Expects tensor shape (1, 3, H, W).
        """
        tensor = tensor.clamp(-1, 1)
        tensor = (tensor + 1) / 2  # Scale to [0,1]
        tensor = tensor.squeeze(0)  # (3, H, W)
        tensor = tensor.permute(1, 2, 0)  # (H, W, 3)
        array = (tensor.cpu().numpy() * 255).astype(np.uint8)
        return Image.fromarray(array, mode="RGB")

#########################################
# Data Collection Pipeline with Self-Assessment
#########################################
class DataCollectionPipeline:
    def __init__(self, diffusion_model: DiffusionSimulator, config: Config, nima_model: NIMA):
        self.model = diffusion_model
        self.config = config
        self.nima_model = nima_model

    # def collect_single_sample(self, latent: torch.Tensor, prompt: str) -> dict:
    #     """
    #     Runs a single generation, computes quality using the NIMA model, and assigns a label.
    #     """
    #     generated_image, noise_sequence = self.model.reverse_diffusion(latent, prompt=prompt)
    #     quality_score = compute_nima_quality(generated_image, self.nima_model, self.config)
    #     # Assign label: 1 if quality_score meets or exceeds threshold, else 0.
    #     label = 1 if quality_score >= self.config.quality_threshold else 0

    #     sample_data = {
    #         "prompt": prompt,
    #         "noise_sequence": [n.cpu().numpy().tolist() for n in noise_sequence],
    #         "generated_image": np.array(generated_image),
    #         "quality_score": quality_score,
    #         "label": label
    #     }
    #     return sample_data

    def collect_single_sample(self, latent: torch.Tensor, prompt: str) -> dict:
        """
        Runs a single generation, computes quality using the NIMA model, and assigns a label.
        """
        generated_image, noise_sequence = self.model.reverse_diffusion(latent, prompt=prompt)
        quality_score = compute_nima_quality(generated_image, self.nima_model, self.config)
        
        # Assign label: 1 if quality_score meets or exceeds threshold, else 0.
        label = 1 if quality_score >= self.config.quality_threshold else 0
    
        # Stack noise_sequence tensors into a single NumPy array of shape (num_timesteps, 3, 64, 64)
        noise_sequence_np = torch.stack(noise_sequence).cpu().numpy()
    
        # Convert generated PIL image to NumPy array
        generated_image_np = np.array(generated_image)
    
        sample_data = {
            "prompt": prompt,
            "noise_sequence": noise_sequence_np,  # much more efficient than list of lists
            "generated_image": generated_image_np,
            "quality_score": quality_score,
            "label": label
        }
        return sample_data


    # def run(self, num_samples: int) -> List[dict]:
    #     """
    #     Generates a dataset by iterating over the prompt list (cycling if needed) and collecting samples.
    #     """
    #     dataset = []
    #     prompts = self.config.prompts
    #     for sample_id in range(num_samples):
    #         prompt = prompts[sample_id % len(prompts)]
    #         # Generate an RGB latent tensor with dimensions (1, 3, 64, 64).
    #         latent = torch.randn(1, 3, 64, 64, device=self.config.device)
    #         sample_data = self.collect_single_sample(latent, prompt)
    #         dataset.append(sample_data)
    #         if (sample_id + 1) % 100 == 0:
    #             print(f"Collected {sample_id + 1} samples so far...")
    #     return dataset

    def run_balanced(self, num_samples: int) -> Tuple[List[dict], float]:
        """
        Generates a dataset and dynamically finds a threshold to balance label counts.
        """
        samples = []
        quality_scores = []
        prompts = self.config.prompts
    
        for sample_id in range(num_samples):
            prompt = prompts[sample_id % len(prompts)]
            latent = torch.randn(1, 3, 64, 64, device=self.config.device)
            generated_image, noise_sequence = self.model.reverse_diffusion(latent, prompt=prompt)
            quality_score = compute_nima_quality(generated_image, self.nima_model, self.config)
    
            sample = {
                "prompt": prompt,
                "noise_sequence": torch.stack(noise_sequence).cpu().numpy(),
                "generated_image": np.array(generated_image),
                "quality_score": quality_score  # temporarily store without label
            }
            samples.append(sample)
            quality_scores.append(quality_score)
    
            if (sample_id + 1) % 100 == 0:
                print(f"Generated {sample_id + 1} samples...")
    
        # Find median quality score as dynamic threshold for balance
        scores_np = np.array(quality_scores)
        dynamic_threshold = float(np.median(scores_np))
        print(f"Dynamic threshold selected: {dynamic_threshold:.4f}")
    
        # Assign labels based on dynamic threshold
        for sample in samples:
            sample["label"] = 1 if sample["quality_score"] >= dynamic_threshold else 0
    
        # Optional: verify balance
        labels = [s["label"] for s in samples]
        print(f"Label counts: 0 → {labels.count(0)}, 1 → {labels.count(1)}")
    
        return samples, dynamic_threshold


#########################################
# Main Script
# #########################################
# if __name__ == "__main__":
#     # Initialize configuration, diffusion simulator, and NIMA model.
#     config = Config()
#     diffusion_model = DiffusionSimulator(config)
#     nima_model = load_nima_model(config)
#     pipeline = DataCollectionPipeline(diffusion_model, config, nima_model)

#     # Number of samples to generate.
#     num_samples_to_generate = 1000
#     dataset = pipeline.run(num_samples_to_generate)

#     # Save the dataset to disk in NPZ format.
#     dataset_path = os.path.join(config.output_dir, "seq2classification_dataset.npz")
#     np.savez_compressed(dataset_path, dataset=dataset)
#     print(f"Dataset saved to {dataset_path}")

if __name__ == "__main__":
    config = Config()
    diffusion_model = DiffusionSimulator(config)
    nima_model = load_nima_model(config)
    pipeline = DataCollectionPipeline(diffusion_model, config, nima_model)

    num_samples_to_generate = 1000
    dataset, dynamic_thresh = pipeline.run_balanced(num_samples_to_generate)

    # Save dataset (without quality_score if not needed)
    for sample in dataset:
        sample.pop("quality_score", None)  # Optional: clean up if not needed

    dataset_path = os.path.join(config.output_dir, "seq2classification_dataset.npz")
    np.savez_compressed(dataset_path, dataset=dataset)
    print(f"Balanced dataset saved to {dataset_path}")

In [ ]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Assuming you already have this:
# from your_dataset_file import DiffusionSequenceDataset
dataset = DiffusionSequenceDataset("collected_data_large/seq2classification_dataset.npz")
loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Count label distribution
labels = [label for _, label in dataset]
class_0 = labels.count(0)
class_1 = labels.count(1)
print(f"Label distribution: [0]: {class_0}, [1]: {class_1}")

# ===============================
# Simple CNN-RNN + Linear Classifier
# ===============================
# class CNN_RNN_Classifier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.cnn = nn.Sequential(
#             nn.Conv2d(3, 32, kernel_size=3, padding=1),  # input channels = 3
#             nn.ReLU(),
#             nn.MaxPool2d(2),  # 32x32
#             nn.Conv2d(32, 64, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(2),  # 16x16
#             nn.AdaptiveAvgPool2d((1, 1))
#         )
#         self.rnn = nn.GRU(input_size=64, hidden_size=128, batch_first=True)
#         self.fc = nn.Linear(128, 1)

#     def forward(self, x):
#         B, T, C, H, W = x.size()
#         x = x.view(B * T, C, H, W)         # [B*T, C, H, W]
#         x = self.cnn(x)                    # [B*T, 64, 1, 1]
#         x = x.view(B, T, -1)               # [B, T, 64]
#         _, h_n = self.rnn(x)               # [1, B, 128]
#         h_n = h_n.squeeze(0)               # [B, 128]
#         return self.fc(h_n)                # [B, 1] (raw logits)

class ImprovedCNN_RNN_Classifier(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        # More sophisticated CNN feature extractor
        self.cnn = nn.Sequential(
            # First block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32
            
            # Second block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16
            
            # Third block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 8x8
            
            nn.AdaptiveAvgPool2d((1, 1))  # Global pooling
        )
        
        # Bidirectional GRU for better temporal modeling
        self.rnn = nn.GRU(
            input_size=128,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=dropout_rate if dropout_rate > 0 and 2 > 1 else 0,
            bidirectional=True
        )
        
        # Fully connected layers with dropout
        self.fc = nn.Sequential(
            nn.Linear(256*2, 128),  # *2 for bidirectional
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)
        )
        
    def forward(self, x):
        B, T, C, H, W = x.size()
        
        # CNN feature extraction
        x = x.view(B * T, C, H, W)
        x = self.cnn(x)  # [B*T, 128, 1, 1]
        x = x.view(B, T, -1)  # [B, T, 128]
        
        # RNN sequence processing
        output, _ = self.rnn(x)  # output shape: [B, T, 256*2]
        
        # Get final time step output
        final_hidden = output[:, -1, :]  # [B, 256*2]
        
        # Classification
        return self.fc(final_hidden)  # [B, 1]
# import torch.optim as optim

# # Move model to device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = BetterQualityMLP(input_dim=10).to(device)  # Make sure input_dim matches your data

# # Loss & optimizer
# loss_fn = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# # Add this before the training loop to inspect the data shape
# for noise_seq, label in loader:
#     print(f"Noise sequence shape: {noise_seq.shape}")
#     print(f"Label shape: {label.shape}")
#     break
# # === Training Loop ===
# for epoch in range(10):
#     model.train()
#     total_loss = 0
#     for noise_seq, label in loader:
#         noise_seq = noise_seq.to(device).float()  # Shape: [B, input_dim]
#         label = label.unsqueeze(1).float().to(device)  # Shape: [B, 1]

#         pred = model(noise_seq)
#         loss = loss_fn(pred, label)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}, Loss: {total_loss / len(loader):.4f}")

# # ===============================
# # Evaluation
# # ===============================
# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for noise_seq, label in loader:
#         noise_seq = noise_seq.to(device)
#         label = label.to(device).float()

#         logits = model(noise_seq)
#         probs = torch.sigmoid(logits)
#         preds = (probs > 0.5).long().squeeze()

#         all_preds.extend(preds.cpu().numpy())
#         all_labels.extend(label.cpu().numpy())


# Move model to device
# Hyperparameters
EPOCHS = 20
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.4
BATCH_SIZE = 8  # Keep as is or adjust if memory allows

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedCNN_RNN_Classifier(dropout_rate=DROPOUT_RATE).to(device)

# Get class weights for balanced learning
num_samples = len(dataset)
class_0_weight = num_samples / (2 * class_0)
class_1_weight = num_samples / (2 * class_1)
class_weights = torch.tensor([class_0_weight, class_1_weight], device=device)

# Loss & optimizer with weight decay for regularization
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([class_1_weight/class_0_weight], device=device))
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

# Training loop with validation
best_val_f1 = 0
best_model_state = None

for epoch in range(EPOCHS):
    # Training phase
    model.train()
    total_loss = 0
    all_train_preds = []
    all_train_labels = []
    
    for noise_seq, label in loader:
        noise_seq = noise_seq.to(device).float() / 255.0  # Normalize to [0,1]
        label = label.unsqueeze(1).float().to(device)
        
        # Forward pass
        pred = model(noise_seq)
        loss = loss_fn(pred, label)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # Track metrics
        total_loss += loss.item()
        probs = torch.sigmoid(pred)
        train_preds = (probs > 0.5).long().squeeze()
        all_train_preds.extend(train_preds.cpu().numpy())
        all_train_labels.extend(label.squeeze().cpu().numpy())
    
    # Training metrics
    train_acc = accuracy_score(all_train_labels, all_train_preds)
    train_f1 = f1_score(all_train_labels, all_train_preds)
    
    # Update learning rate based on loss
    avg_loss = total_loss / len(loader)
    scheduler.step(avg_loss)
    
    # Print training metrics
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
    
    # Validation phase - using training data as validation for this example
    # In a real scenario, you would have a separate validation loader
    model.eval()
    all_val_preds = []
    all_val_labels = []
    
    with torch.no_grad():
        for noise_seq, label in loader:  # Using same loader for demonstration
            noise_seq = noise_seq.to(device).float() / 255.0
            label = label.to(device).float()
            
            logits = model(noise_seq)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).long().squeeze()
            
            all_val_preds.extend(preds.cpu().numpy())
            all_val_labels.extend(label.cpu().numpy())
    
    # Calculate validation metrics
    val_acc = accuracy_score(all_val_labels, all_val_preds)
    val_f1 = f1_score(all_val_labels, all_val_preds)
    print(f"Validation - Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
    
    # Save best model
    # Save best model with F1 below 0.9
    if val_f1 > best_val_f1 and val_f1 < 0.9:
        best_val_f1 = val_f1
        best_model_state = model.state_dict().copy()
        print(f"New best model saved with F1: {best_val_f1:.4f}")

# Load best model for final evaluation
if best_model_state is not None:
    model.load_state_dict(best_model_state)

# Final evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for noise_seq, label in loader:
        noise_seq = noise_seq.to(device).float() / 255.0
        label = label.to(device).float()
        
        logits = model(noise_seq)
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long().squeeze()
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

# Calculate final metrics
final_acc = accuracy_score(all_labels, all_preds)
final_f1 = f1_score(all_labels, all_preds)
print(f"Final Evaluation - Accuracy: {final_acc:.4f}, F1 Score: {final_f1:.4f}")

In [ ]:
import pickle 

with open('model.pkl','wb') as file:
    pickle.dump(model, file)

In [ ]:
import torch
import torch.nn as nn

class ImprovedLatentCNN_RNN_Classifier(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        # Explicitly use 4 input channels for SD latents
        self.cnn = nn.Sequential(
            # First block - explicitly using 4 channel latent input
            nn.Conv2d(4, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32
            
            # Second block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16
            
            # Third block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 8x8
            
            nn.AdaptiveAvgPool2d((1, 1))  # Global pooling
        )
        
        # Bidirectional GRU for better temporal modeling
        self.rnn = nn.GRU(
            input_size=128,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=dropout_rate if dropout_rate > 0 and 2 > 1 else 0,
            bidirectional=True
        )
        
        # Fully connected layers with dropout
        self.fc = nn.Sequential(
            nn.Linear(256*2, 128),  # *2 for bidirectional
            nn.BatchNorm1d(128),    # Correctly use BatchNorm1d here
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)
        )
        
    def forward(self, x):
        # Process the batch dimension and sequence length appropriately
        if len(x.shape) == 5:  # [B, T, C, H, W]
            B, T, C, H, W = x.size()
            # Reshape for CNN processing
            x = x.reshape(B * T, C, H, W)
        else:  # Handle the case where input might be [B, C, H, W]
            B = x.size(0)
            T = 1
            x = x  # No reshape needed
        
        # CNN feature extraction
        x = self.cnn(x)  # [B*T, 128, 1, 1]
        x = x.reshape(B, T, -1) if T > 1 else x.reshape(B, -1).unsqueeze(1)  # [B, T, 128]
        
        # RNN sequence processing
        output, _ = self.rnn(x)  # output shape: [B, T, 256*2]
        
        # Get final time step output
        final_hidden = output[:, -1, :]  # [B, 256*2]
        
        # Classification
        return self.fc(final_hidden)  # [B, 1]

# Modified classification function that handles the batched latent frames correctly
def classify_with_latent_model(classifier, latent_frames, device):
    """
    Classify latent noise frames with the CNN-RNN classifier
    
    Args:
        classifier: The CNN-RNN classifier model
        latent_frames: List of latent tensors or tensor with shape [T, 1, 4, 64, 64] or [1, T, 4, 64, 64]
        device: Device to run classification on
    
    Returns:
        prediction: Binary prediction (0 or 1)
        probability: Confidence score (0 to 1)
    """
    # Proper tensor preparation based on input format
    if isinstance(latent_frames, list):
        # Convert list of latents to proper tensor format
        latent_batch = torch.stack(latent_frames, dim=0)  # [T, 1, 4, 64, 64]
        # Take only the first dimension from the second axis (batch dimension)
        latent_batch = latent_batch[:, 0:1, :, :, :]
        # Reshape to [1, T, 4, 64, 64] (batch dim first)
        latent_batch = latent_batch.permute(1, 0, 2, 3, 4)
    else:
        # Already a tensor, ensure correct shape
        if latent_frames.dim() == 5:
            if latent_frames.shape[0] > latent_frames.shape[1]:  # [T, B, C, H, W]
                latent_batch = latent_frames.permute(1, 0, 2, 3, 4)  # -> [B, T, C, H, W]
            else:
                latent_batch = latent_frames  # Already [B, T, C, H, W]
        else:
            # Handle unexpected tensor shapes
            raise ValueError(f"Unexpected latent shape: {latent_frames.shape}")
    
    # Debug information to check tensor shape and channels
    print(f"Debug - latent_batch shape before CNN: {latent_batch.shape}")
    
    # Move to device and ensure float type
    latent_batch = latent_batch.to(device).float()
    
    with torch.no_grad():
        logits = classifier(latent_batch)
        probs = torch.sigmoid(logits)
        prediction = (probs > 0.5).long().item()
    
    return prediction, probs.item()

# Main function to generate images with the improved classifier
def generate_with_improved_classifier(prompt, model_ckpt_path=None, num_inference_steps=50, guidance_scale=7.5):
    from diffusers import StableDiffusionPipeline, DDIMScheduler
    from PIL import Image
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load Stable Diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        use_safetensors=True
    ).to(device)

    # Load DDIM scheduler
    pipe.scheduler = DDIMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    pipe.scheduler.set_timesteps(num_inference_steps)
    timesteps = pipe.scheduler.timesteps

    # Get text embeddings with classifier-free guidance
    text_embeddings = pipe._encode_prompt(
        prompt,
        device,
        1,
        do_classifier_free_guidance=True,
    )

    # Create the initial latent noise
    # Use pipe.unet.config.in_channels instead of pipe.unet.in_channels to avoid deprecation warning
    latents = torch.randn(
        (1, pipe.unet.config.in_channels, 64, 64),
        generator=None,
        device=device,
        dtype=pipe.unet.dtype
    )
    latents = latents * pipe.scheduler.init_noise_sigma

    # Create and initialize the improved classifier
    classifier = ImprovedLatentCNN_RNN_Classifier().to(device)
    classifier.eval()
    
    # Load checkpoint if provided
    if model_ckpt_path:
        try:
            checkpoint = torch.load(model_ckpt_path, map_location=device)
            classifier.load_state_dict(checkpoint)
            print("✅ Successfully loaded latent classifier checkpoint")
        except Exception as e:
            print(f"❌ Error loading latent classifier checkpoint: {e}")
            print("⚠ Continuing with untrained classifier")

    # For storing latent frames
    latent_frames = []
    # To keep snapshots of latents after each accepted step
    accepted_latent_snapshots = []
    rollback_limit = 10  # limit the number of rollbacks
    step_results = []

    print(f"\n🎨 Starting diffusion with prompt: \"{prompt}\"")
    print(f"📈 Number of timesteps: {len(timesteps)}")

    good_steps = 0   # counts accepted (good) steps
    current_idx = 0  # index into the scheduler's timesteps

    # We will run until all steps are complete
    while current_idx < len(timesteps):
        t = timesteps[current_idx]
        # Save the current latent state (snapshot) so we can revert if necessary
        accepted_latent_snapshots.append(latents.clone())

        # Store the current latent for classification
        latent_frames.append(latents.detach().clone())

        # Expand the latents for classifier-free guidance
        latent_model_input = torch.cat([latents] * 2) if guidance_scale > 1.0 else latents
        
        # Predict the noise residual with the UNet
        with torch.no_grad():
            noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
        
        # Perform guidance
        if guidance_scale > 1.0:
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

        # Only classify if we have enough frames for temporal context
        prediction = 1  # Default to accepting the step
        prob = 1.0
        
        if len(latent_frames) >= 3:
            try:
                # Use only the last 50 frames max to avoid memory issues
                recent_frames = latent_frames[-50:]
                
                # Use the classify_with_latent_model function with our frames
                prediction, prob = classify_with_latent_model(classifier, recent_frames, device)
                step_results.append((current_idx, prediction, prob))
            except Exception as e:
                print(f"⚠ Classification error: {e}")
                # Continue with default prediction
        
        status = 'GOOD ✅' if prediction == 1 else 'FAIL ❌'            
        print(f"  Step {current_idx+1}/{len(timesteps)}: {status} (confidence: {prob:.4f})")

        if prediction == 0:
            if rollback_limit > 0 and current_idx > 0:
                rollback_limit -= 1
                # Revert to the previous accepted latent state
                latents = accepted_latent_snapshots[current_idx - 1].clone()
                latent_frames.pop()  # remove the current (failed) latent frame
                print(f"  🔁 Rolling back from step {current_idx+1} (Rollback limit now: {rollback_limit})")
                # Do not increment current_idx so that we retry this timestep
                continue
            else:
                print("  🚫 Rollback limit reached or no previous good step available; accepting FAIL step.")
        
        # If the step is GOOD (or forced accepted), update latents normally:
        latents = pipe.scheduler.step(noise_pred, t, latents).prev_sample
        good_steps += 1
        current_idx += 1

    # After diffusion, decode the final latent to an image
    try:
        with torch.no_grad():
            # Convert latents to the VAE's dtype
            latents_for_image = latents.to(pipe.vae.dtype)
            latents_for_image = 1 / 0.18215 * latents_for_image  # Scale for VAE
            decoded = pipe.vae.decode(latents_for_image).sample
            image = (decoded / 2 + 0.5).clamp(0, 1)
            image = (image * 255).type(torch.uint8)
            image = image.cpu().permute(0, 2, 3, 1)
            final_image = Image.fromarray(image[0].numpy())
            
            # Save the image
            final_image.save("generated_image.png")
            print("Image saved as 'generated_image.png'")
            
            return final_image, latent_frames, step_results
    except Exception as e:
        print(f"❌ Error generating the final image: {e}")
        return None, latent_frames, step_results

# Example usage
if __name__ == "__main__":
    prompt = "A deer standing in middle of misty forest"
    model_ckpt_path = None  # No checkpoint needed for testing
    
    # Generate with improved latent noise classifier guidance
    image, frames, results = generate_with_improved_classifier(
        prompt, 
        model_ckpt_path,
        num_inference_steps=50,
        guidance_scale=7.5
    )

In [ ]:
image

In [ ]:
import torch
import numpy as np
from PIL import Image
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from diffusers import StableDiffusionPipeline, DDIMScheduler
import torch.nn as nn
import matplotlib.pyplot as plt
import time
from io import BytesIO

# Define the ImprovedLatentCNN_RNN_Classifier
class ImprovedLatentCNN_RNN_Classifier(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        # Explicitly use 4 input channels for SD latents
        self.cnn = nn.Sequential(
            # First block - explicitly using 4 channel latent input
            nn.Conv2d(4, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32

            # Second block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16

            # Third block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 8x8

            nn.AdaptiveAvgPool2d((1, 1))  # Global pooling
        )

        # Bidirectional GRU for better temporal modeling
        self.rnn = nn.GRU(
            input_size=128,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=dropout_rate if dropout_rate > 0 and 2 > 1 else 0,
            bidirectional=True
        )

        # Fully connected layers with dropout
        self.fc = nn.Sequential(
            nn.Linear(256*2, 128),  # *2 for bidirectional
            nn.BatchNorm1d(128),    # Correctly use BatchNorm1d here
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        # Process the batch dimension and sequence length appropriately
        if len(x.shape) == 5:  # [B, T, C, H, W]
            B, T, C, H, W = x.size()
            # Reshape for CNN processing
            x = x.reshape(B * T, C, H, W)
        else:  # Handle the case where input might be [B, C, H, W]
            B = x.size(0)
            T = 1
            x = x  # No reshape needed

        # CNN feature extraction
        x = self.cnn(x)  # [B*T, 128, 1, 1]
        x = x.reshape(B, T, -1) if T > 1 else x.reshape(B, -1).unsqueeze(1)  # [B, T, 128]

        # RNN sequence processing
        output, _ = self.rnn(x)  # output shape: [B, T, 256*2]

        # Get final time step output
        final_hidden = output[:, -1, :]  # [B, 256*2]

        # Classification
        return self.fc(final_hidden)  # [B, 1]


def load_neg_generator():
    """
    Load the negative prompt generator model
    """
    try:
        tokenizer = AutoTokenizer.from_pretrained("CharanSaiVaddi/negopt-sft-model")
        model = AutoModelForSeq2SeqLM.from_pretrained("CharanSaiVaddi/negopt-sft-model")
        return tokenizer, model
    except Exception as e:
        print(f"Error loading negative prompt generator: {e}")
        return None, None


def load_stable_diffusion():
    """
    Load the Stable Diffusion pipeline
    """
    try:
        # Determine device
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Load pipeline
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            use_safetensors=True
        ).to(device)
        
        # Use DDIM scheduler
        pipe.scheduler = DDIMScheduler.from_pretrained(
            "runwayml/stable-diffusion-v1-5", 
            subfolder="scheduler"
        )
        
        return pipe, device
    except Exception as e:
        print(f"Error loading Stable Diffusion: {e}")
        return None, None


def load_classifier():
    """
    Load the latent classifier model
    """
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        classifier = ImprovedLatentCNN_RNN_Classifier().to(device)
        classifier.eval()
        return classifier
    except Exception as e:
        print(f"Error loading classifier: {e}")
        return None


@torch.no_grad()
def generate_negative(
    prompt: str,
    tokenizer: AutoTokenizer,
    model: AutoModelForSeq2SeqLM,
    max_length: int = 64,
    num_beams: int = 4
) -> str:
    """
    Generates an optimized negative prompt for the given positive `prompt`.
    """
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True
        )
        neg_prompt = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return neg_prompt
    except Exception as e:
        print(f"Error generating negative prompt: {e}")
        return ""


def classify_with_latent_model(classifier, latent_frames, device):
    """
    Classify latent noise frames with the CNN-RNN classifier
    """
    try:
        # Proper tensor preparation based on input format
        if isinstance(latent_frames, list):
            # Convert list of latents to proper tensor format
            latent_batch = torch.stack(latent_frames, dim=0)  # [T, 1, 4, 64, 64]
            # Take only the first dimension from the second axis (batch dimension)
            latent_batch = latent_batch[:, 0:1, :, :, :]
            # Reshape to [1, T, 4, 64, 64] (batch dim first)
            latent_batch = latent_batch.permute(1, 0, 2, 3, 4)
        else:
            # Already a tensor, ensure correct shape
            if latent_frames.dim() == 5:
                if latent_frames.shape[0] > latent_frames.shape[1]:  # [T, B, C, H, W]
                    latent_batch = latent_frames.permute(1, 0, 2, 3, 4)  # -> [B, T, C, H, W]
                else:
                    latent_batch = latent_frames  # Already [B, T, C, H, W]
            else:
                # Handle unexpected tensor shapes
                raise ValueError(f"Unexpected latent shape: {latent_frames.shape}")

        # Move to device and ensure float type
        latent_batch = latent_batch.to(device).float()

        with torch.no_grad():
            logits = classifier(latent_batch)
            probs = torch.sigmoid(logits)
            prediction = (probs > 0.5).long().item()

        return prediction, probs.item()
    except Exception as e:
        print(f"Error in classification: {e}")
        return 1, 1.0  # Default to accepting


def advanced_image_generation(
    prompt, 
    pipe, 
    device, 
    classifier, 
    negative_prompt=None, 
    use_classifier_guidance=True,
    num_inference_steps=50, 
    guidance_scale=7.5,
    verbose=True
):
    """
    Generate image with advanced techniques (negative prompts + classifier guidance)
    """
    # Setup scheduler
    pipe.scheduler.set_timesteps(num_inference_steps)
    timesteps = pipe.scheduler.timesteps
    
    if verbose:
        print(f"🎨 Starting diffusion with prompt: \"{prompt}\"")
        if negative_prompt:
            print(f"🎨 Using negative prompt: \"{negative_prompt}\"")
    
    # Get text embeddings with classifier-free guidance
    text_embeddings = pipe._encode_prompt(
        prompt,
        device,
        1,
        do_classifier_free_guidance=True,
        negative_prompt=negative_prompt
    )
    
    # Create the initial latent noise
    latents = torch.randn(
        (1, pipe.unet.config.in_channels, 64, 64),
        generator=None,
        device=device,
        dtype=pipe.unet.dtype
    )
    latents = latents * pipe.scheduler.init_noise_sigma
    
    # Setup for latent frames and tracking
    latent_frames = []
    accepted_latent_snapshots = []
    rollback_limit = 5  # limit the number of rollbacks
    step_results = []
    
    good_steps = 0
    current_idx = 0
    
    # Step visualization data
    step_indices = []
    confidences = []
    decisions = []
    
    # Generation loop
    while current_idx < len(timesteps):
        t = timesteps[current_idx]
        # Progress updates
        progress = (current_idx + 1) / len(timesteps)
        if verbose:
            print(f"Progress: {progress*100:.1f}%", end="\r")
        
        # Save the current latent state
        accepted_latent_snapshots.append(latents.clone())
        
        # Store the current latent for classification
        latent_frames.append(latents.detach().clone())
        
        # Expand the latents for classifier-free guidance
        latent_model_input = torch.cat([latents] * 2) if guidance_scale > 1.0 else latents
        
        # Predict the noise residual
        with torch.no_grad():
            noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
        
        # Perform guidance
        if guidance_scale > 1.0:
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
        
        # Classification and decision
        prediction = 1  # Default to accepting
        prob = 1.0
        
        if use_classifier_guidance and len(latent_frames) >= 3 and classifier is not None:
            try:
                # Use only the last 50 frames max
                recent_frames = latent_frames[-50:]
                prediction, prob = classify_with_latent_model(classifier, recent_frames, device)
                step_results.append((current_idx, prediction, prob))
                
                # Store data for visualization
                step_indices.append(current_idx)
                confidences.append(prob)
                decisions.append(prediction)
            except Exception as e:
                if verbose:
                    print(f"⚠️ Classification error: {e}")
        
        status = 'GOOD ✅' if prediction == 1 else 'FAIL ❌'
        if verbose:
            print(f"Step {current_idx+1}/{len(timesteps)}: {status} (confidence: {prob:.4f})")
        
        # Handle rejected steps
        if prediction == 0 and use_classifier_guidance:
            if rollback_limit > 0 and current_idx > 0:
                rollback_limit -= 1
                # Revert to previous accepted state
                latents = accepted_latent_snapshots[current_idx - 1].clone()
                latent_frames.pop()  # remove failed frame
                if verbose:
                    print(f"🔁 Rolling back from step {current_idx+1} (Rollbacks left: {rollback_limit})")
                continue
            else:
                if verbose:
                    print("🚫 Rollback limit reached; accepting anyway")
        
        # Update latents
        latents = pipe.scheduler.step(noise_pred, t, latents).prev_sample
        good_steps += 1
        current_idx += 1
    
    # Decode the final latent to an image
    try:
        with torch.no_grad():
            latents_for_image = latents.to(pipe.vae.dtype)
            latents_for_image = 1 / 0.18215 * latents_for_image
            decoded = pipe.vae.decode(latents_for_image).sample
            image = (decoded / 2 + 0.5).clamp(0, 1)
            image = (image * 255).type(torch.uint8)
            image = image.cpu().permute(0, 2, 3, 1)
            final_image = Image.fromarray(image[0].numpy())
            
            # Create and return statistics
            stats = {
                "total_steps": len(step_results),
                "accepted_steps": sum(1 for _, pred, _ in step_results if pred == 1),
                "rejected_steps": sum(1 for _, pred, _ in step_results if pred == 0),
                "step_details": step_results,
                "confidence_values": list(zip(step_indices, confidences))
            }
            
            return final_image, stats
    except Exception as e:
        if verbose:
            print(f"❌ Error generating final image: {e}")
        return None, {"error": str(e)}


def visualize_results(image, stats, save_path=None):
    """
    Visualize the generation results
    """
    plt.figure(figsize=(15, 10))
    
    # Plot the image
    plt.subplot(2, 2, 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Generated Image')
    
    # Plot step decisions
    if "step_details" in stats:
        step_indices = [step for step, _, _ in stats["step_details"]]
        decisions = [pred for _, pred, _ in stats["step_details"]]
        confidences = [conf for _, _, conf in stats["step_details"]]
        
        plt.subplot(2, 2, 2)
        colors = ['red' if d == 0 else 'green' for d in decisions]
        plt.bar(step_indices, [1] * len(step_indices), color=colors)
        plt.ylim(0, 1.2)
        plt.title('Step Decisions (Green=Accept, Red=Reject)')
        plt.xlabel('Diffusion Step')
        
        # Plot confidence scores
        plt.subplot(2, 2, 3)
        plt.plot(step_indices, confidences, marker='o', color='blue')
        plt.ylim(0, 1.05)
        plt.title('Confidence Scores')
        plt.xlabel('Diffusion Step')
        plt.ylabel('Confidence')
        plt.axhline(y=0.5, color='r', linestyle='--')
        
        # Print statistics
        plt.subplot(2, 2, 4)
        plt.axis('off')
        stats_text = f"""
        Generation Statistics:
        - Total Steps: {stats.get('total_steps', 'N/A')}
        - Accepted Steps: {stats.get('accepted_steps', 'N/A')}
        - Rejected Steps: {stats.get('rejected_steps', 'N/A')}
        """
        plt.text(0.1, 0.5, stats_text, fontsize=12)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path)
        print(f"Results visualization saved to {save_path}")
    
    plt.show()


def save_image(image, file_path):
    """
    Save the generated image to a file
    """
    image.save(file_path)
    print(f"Image saved to {file_path}")
    return file_path


def generate_image_from_prompt(
    prompt, 
    neg_prompt_option="auto",
    custom_neg_prompt=None,
    use_classifier=True,
    steps=50,
    guidance=7.5,
    visualize=True,
    save_image_path=None,
    save_viz_path=None,
    verbose=True
):
    """
    Main function to generate an image from a prompt
    """
    # Load models
    if verbose:
        print("Loading models...")
    
    neg_tokenizer, neg_model = load_neg_generator()
    pipe, device = load_stable_diffusion()
    classifier = load_classifier() if use_classifier else None
    
    if pipe is None:
        print("❌ Failed to load Stable Diffusion. Cannot generate image.")
        return None, None
    
    # Handle negative prompt
    negative_prompt = None
    if neg_prompt_option == "auto":
        if neg_tokenizer is not None and neg_model is not None:
            if verbose:
                print("Generating negative prompt...")
            negative_prompt = generate_negative(prompt, neg_tokenizer, neg_model)
            if verbose:
                print(f"Generated negative prompt: {negative_prompt}")
        else:
            print("⚠️ Negative prompt generator not available")
    elif neg_prompt_option == "custom" and custom_neg_prompt:
        negative_prompt = custom_neg_prompt
    
    # Generate image
    if verbose:
        print(f"Generating image for prompt: '{prompt}'")
    
    result_image, stats = advanced_image_generation(
        prompt,
        pipe,
        device,
        classifier,
        negative_prompt,
        use_classifier,
        steps,
        guidance,
        verbose
    )
    
    if result_image:
        if verbose:
            print("✅ Image generation complete!")
        
        # Save image if requested
        if save_image_path:
            save_image(result_image, save_image_path)
        
        # Visualize results if requested
        if visualize:
            visualize_results(result_image, stats, save_viz_path)
            
        return result_image, stats
    else:
        print("❌ Failed to generate image.")
        return None, stats


# Example usage in a Python script or notebook
if __name__ == "__main__":
    # Example prompt
    prompt = input("Enter your prompt: ")
    
    # Generate image
    image, stats = generate_image_from_prompt(
        prompt=prompt,
        neg_prompt_option="auto",  # Options: "none", "auto", "custom"
        custom_neg_prompt="low quality, blurry, distorted",  # Used if neg_prompt_option="custom"
        use_classifier=True,
        steps=50,
        guidance=7.5,
        visualize=True,
        save_image_path="generated_image.png",
        save_viz_path="generation_results.png"
    )

In [ ]:
pip install rouge_score nltk

In [ ]:
!pip install torch torchvision scikit-learn scipy evaluate pytorch-fid torchmetrics transformers

In [ ]:
import os
import numpy as np
import torch
from scipy import stats
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    precision_recall_curve,
    brier_score_loss,
)
from sklearn.calibration import calibration_curve
from pytorch_fid import fid_score  # FID metric
from torchmetrics.image.inception import InceptionScore  # Inception Score metric
from torchmetrics.multimodal import CLIPScore  # CLIPScore metric
import evaluate  # Hugging Face evaluate for BLEU/ROUGE
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from PIL import Image  # Ensure Image is defined

# -----------------------------------------------------------------------------
# 1) Latent CNN-RNN Classifier Evaluation
# -----------------------------------------------------------------------------
def eval_latent_classifier(y_true, y_pred_logits):
    """
    y_true: list or np.array of 0/1 labels
    y_pred_logits: list or np.array of raw logits from model
    """
    y_prob = torch.sigmoid(torch.tensor(y_pred_logits)).numpy()
    y_pred = (y_prob > 0.5).astype(int)

    # Classification report: precision, recall, F1
    print(classification_report(y_true, y_pred, digits=4))

    # ROC AUC
    auc = roc_auc_score(y_true, y_prob)
    print(f"ROC AUC: {auc:.4f}")

    # Calibration curve & Brier score
    frac_pos, mean_pred = calibration_curve(y_true, y_prob, n_bins=10)
    brier = brier_score_loss(y_true, y_prob)
    print(f"Brier score: {brier:.4f}")
    return {
        "classification_report": classification_report(y_true, y_pred, output_dict=True),
        "roc_auc": auc,
        "calibration": (mean_pred.tolist(), frac_pos.tolist()),
        "brier": brier,
    }

# -----------------------------------------------------------------------------
# 2) Negative-Prompt Generator Evaluation
# -----------------------------------------------------------------------------
# load GPT-2 for perplexity
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# Fix padding token issue for GPT2 tokenizer
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

gpt2.eval()

bleu = evaluate.load("bleu")      # BLEU metric
rouge = evaluate.load("rouge")    # ROUGE metric (requires rouge_score, nltk)
clipscore = CLIPScore()

# Move CLIPScore to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clipscore = clipscore.to(device)

def perplexity(texts):
    # Tokenize with padding now that pad_token is set
    encodings = gpt2_tokenizer(texts, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = gpt2(**encodings, labels=encodings["input_ids"])
    # mean per-token loss → perplexity
    ppl = torch.exp(outputs.loss)
    return ppl.tolist()

def eval_negative_prompts(references, generations, images_for_clip):
    """
    references: list of human-written negative prompts
    generations: list of auto-generated negative prompts
    images_for_clip: list of PIL images corresponding to positive prompts
    """
    # BLEU & ROUGE
    bleu_res = bleu.compute(predictions=generations, references=[[r] for r in references])
    rouge_res = rouge.compute(predictions=generations, references=references)
    # Perplexity (fluency)
    ppl = perplexity(generations)
    # Prepare images for CLIPScore: convert PIL to torch.Tensor (C,H,W) on same device
    img_tensors = []
    for img in images_for_clip:
        arr = np.array(img)
        tensor = torch.tensor(arr).permute(2,0,1).float() / 255.0
        img_tensors.append(tensor.to(device))
    # Embedding-based CLIPScore: measure how well images avoid unwanted content
    clip_res_tensor = clipscore(images=img_tensors, text=generations)
    # clip_res_tensor is a zero-dim tensor, convert to python float
    clip_score_val = clip_res_tensor.item()
    return {
        "bleu": bleu_res,
        "rouge": rouge_res,
        "perplexity": np.mean(ppl),
        "clipscore": clip_res["clip_score"].item(),
    }

# -----------------------------------------------------------------------------
# 3) Image Generation Metrics
# -----------------------------------------------------------------------------
def eval_generated_images(real_dir, fake_dir, prompts, fake_images, device=device):
    """
    real_dir: path to folder of real images
    fake_dir: path to folder where fake images are saved
    prompts: list of text prompts
    fake_images: list of PIL images generated
    """
    # Save fake images to disk for FID
    os.makedirs(fake_dir, exist_ok=True)
    for i, img in enumerate(fake_images):
        img.save(os.path.join(fake_dir, f"{i:04d}.png"))

    # 3.1 FID
    fid_value = fid_score.calculate_fid_given_paths([real_dir, fake_dir], batch_size=32, device=device)

    # 3.2 Inception Score
    is_metric = InceptionScore()
    is_metric = is_metric.to(device)
    fake_tensors = torch.stack([
        torch.tensor(np.array(img)).permute(2,0,1)
        for img in fake_images
    ]).float() / 255.0
    fake_tensors = fake_tensors.to(device)
    is_res = is_metric(fake_tensors)

    # 3.3 CLIPScore: convert PIL to torch.Tensor
    img_tensors = [
        torch.tensor(np.array(img)).permute(2,0,1).float()/255.0 for img in fake_images
    ]
    img_tensors = [t.to(device) for t in img_tensors]
    clip_res = clipscore(images=img_tensors, text=prompts)

    return {
        "FID": fid_value,
        "InceptionScore": {"mean": is_res["inception_score"].item(), "std": is_res["inception_score_std"].item()},
        "CLIPScore": clip_res["clip_score"].item(),
    }

# -----------------------------------------------------------------------------
# 4) Ablation & Statistical Significance
# -----------------------------------------------------------------------------
def ablation_ttest(metric_a, metric_b):
    """
    Paired t-test between two metric arrays (e.g. FID with vs. without classifier)
    """
    t_stat, p_val = stats.ttest_rel(metric_a, metric_b)
    print(f"Paired t-test: t={t_stat:.4f}, p={p_val:.4f}")
    return {"t_stat": t_stat, "p_value": p_val}

# -----------------------------------------------------------------------------
# Example usage
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # 1) Classifier
    y_true = np.random.randint(0,2,100)
    y_logits = np.random.randn(100)
    cls_results = eval_latent_classifier(y_true, y_logits)

    # 2) Negative prompts (dummy)
    refs = ["low quality", "blurry", "bad anatomy"] * 10
    gens = ["blurry, low detail"] * 30
    neg_results = eval_negative_prompts(refs, gens, images_for_clip=[Image.new("RGB",(64,64))]*30)

    # 3) Image metrics (dummy directories & data)
    real_dir = "data/real"
    fake_dir = "data/fake"
    prompts = ["a cat on a sofa"]*10
    fake_images = [Image.new("RGB",(256,256),color=(i*20, i*20, i*20)) for i in range(10)]
    img_results = eval_generated_images(real_dir, fake_dir, prompts, fake_images)

    # 4) Ablation: compare two FID runs
    fid_a = np.random.randn(10)*5 + 50
    fid_b = fid_a - (np.random.rand(10)*2)
    ablation = ablation_ttest(fid_a, fid_b)

    print("=== RESULTS ===")
    print("Classifier:", cls_results)
    print("NegPrompt:", neg_results)
    print("Image:", img_results)
    print("Ablation:", ablation)


In [ ]:
!pip install --upgrade torch --index-url https://download.pytorch.org/whl/cpu

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss
import pandas as pd

# Dummy dataset class for latent sequences\ n
class LatentSequenceDataset(Dataset):
    def __init__(self, latents, labels):
        # latents: Tensor [N, T, C, H, W]
        # labels: array [N]
        self.latents = latents
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.latents[idx], self.labels[idx]

# Simple Vanilla CNN (no RNN)
class VanillaCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(4, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        # x: [B, T, C, H, W] or [B, C, H, W]
        if x.ndim == 5:
            # average over time
            x = x.mean(dim=1)  # [B, C, H, W]
        out = self.cnn(x).view(x.size(0), -1)
        return self.fc(out).squeeze(1)

# Improved CNN-RNN classifier
class ImprovedLatentCNN_RNN_Classifier(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(4, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )
        self.rnn = nn.GRU(128, 256, num_layers=2, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(512, 128), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B*T, C, H, W)
        feats = self.cnn(x).view(B, T, -1)
        out, _ = self.rnn(feats)
        final = out[:, -1]
        return self.fc(final).squeeze(1)

# Generate synthetic data
N, T, C, H, W = 200, 10, 4, 64, 64
latents = torch.randn(N, T, C, H, W)
labels = torch.randint(0, 2, (N,)).numpy()

# Prepare dataset and split
dataset = LatentSequenceDataset(latents, labels)
train_size = int(0.8 * N)
val_size = N - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)

# Helper to extract features for sklearn
def extract_features(loader, model=None):
    X, y = [], []
    for lat, lbl in loader:
        if model is None:
            # flatten mean-pool
            x = lat.mean(dim=1).view(lat.size(0), -1).numpy()
        else:
            # get CNN features
            with torch.no_grad():
                x_in = lat.mean(dim=1)
                feat = model.cnn(x_in).view(lat.size(0), -1).numpy()
            x = feat
        X.append(x)
        y.append(lbl.numpy())
    return np.concatenate(X), np.concatenate(y)

metrics = {'model': [], 'accuracy': [], 'roc_auc': [], 'brier_score': []}

# 1) Logistic Regression
X_train, y_train = extract_features(train_loader)
X_val, y_val = extract_features(val_loader)
lr = LogisticRegression(max_iter=500).fit(X_train, y_train)
y_prob_lr = lr.predict_proba(X_val)[:,1]
metrics['model'].append('Logistic Regression')
metrics['accuracy'].append(accuracy_score(y_val, y_prob_lr>0.5))
metrics['roc_auc'].append(roc_auc_score(y_val, y_prob_lr))
metrics['brier_score'].append(brier_score_loss(y_val, y_prob_lr))

# 2) SVM
svm = SVC(probability=True).fit(X_train, y_train)
y_prob_svm = svm.predict_proba(X_val)[:,1]
metrics['model'].append('SVM')
metrics['accuracy'].append(accuracy_score(y_val, y_prob_svm>0.5))
metrics['roc_auc'].append(roc_auc_score(y_val, y_prob_svm))
metrics['brier_score'].append(brier_score_loss(y_val, y_prob_svm))

# 3) Vanilla CNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vanilla = VanillaCNN().to(device)
opt = torch.optim.Adam(vanilla.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss()
for epoch in range(5):
    vanilla.train()
    for lat, lbl in train_loader:
        lat, lbl = lat.to(device), lbl.float().to(device)
        logits = vanilla(lat)
        loss = loss_fn(logits, lbl)
        opt.zero_grad(); loss.backward(); opt.step()
vanilla.eval()
y_prob_vanilla, y_true = [], []
with torch.no_grad():
    for lat, lbl in val_loader:
        lat = lat.to(device)
        probs = torch.sigmoid(vanilla(lat)).cpu().numpy()
        y_prob_vanilla.extend(probs)
        y_true.extend(lbl.numpy())
metrics['model'].append('Vanilla CNN')
metrics['accuracy'].append(accuracy_score(y_true, np.array(y_prob_vanilla)>0.5))
metrics['roc_auc'].append(roc_auc_score(y_true, y_prob_vanilla))
metrics['brier_score'].append(brier_score_loss(y_true, y_prob_vanilla))

# 4) Improved CNN-RNN
improved = ImprovedLatentCNN_RNN_Classifier().to(device)
opt2 = torch.optim.Adam(improved.parameters(), lr=1e-3)
for epoch in range(5):
    improved.train()
    for lat, lbl in train_loader:
        lat, lbl = lat.to(device), lbl.float().to(device)
        logits = improved(lat)
        loss = loss_fn(logits, lbl)
        opt2.zero_grad(); loss.backward(); opt2.step()
improved.eval()
y_prob_imp = []
with torch.no_grad():
    for lat, lbl in val_loader:
        lat = lat.to(device)
        probs = torch.sigmoid(improved(lat)).cpu().numpy()
        y_prob_imp.extend(probs)
metrics['model'].append('Improved CNN-RNN')
metrics['accuracy'].append(accuracy_score(y_true, np.array(y_prob_imp)>0.5))
metrics['roc_auc'].append(roc_auc_score(y_true, y_prob_imp))
metrics['brier_score'].append(brier_score_loss(y_true, y_prob_imp))

# Display results
df = pd.DataFrame(metrics)
print(df)

df.to_csv('classifier_benchmark.csv', index=False)
print("Saved results to classifier_benchmark.csv")

In [ ]:
!pip install ace_tools